In [10]:
pip install tqdm sympy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 26.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 21.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import json
from tqdm import tqdm

tokens = {}
total_lines = sum(1 for _ in open("train16.txt", 'r', encoding='utf-8'))
with open("train16.txt", "r", encoding="utf-8") as fin:
    for line in tqdm(fin, total=total_lines, desc="Processing lines"):
        line = line.strip()
        if not line:
            continue
        for token in line.split():
            if token not in tokens:
                tokens[token] = 0
            tokens[token] += 1

print("token dict:", json.dumps(tokens, indent=2))

Processing lines: 100%|██████████| 683536/683536 [00:03<00:00, 196238.85it/s]

token dict: {
  "add": 2196459,
  "mul": 2676318,
  "INT-": 1163047,
  "2": 877693,
  "Y": 469138,
  "x": 1893295,
  "Y'": 731671,
  "a8": 683792,
  "pow": 532528,
  "INT+": 1899959,
  "3": 416770,
  "1": 974956,
  "4": 373808,
  "ln": 254401,
  "exp": 294024,
  "sin": 216166,
  "cos": 227208,
  "sqrt": 164897,
  "tan": 134173,
  "5": 365705,
  "6": 132617,
  "div": 103891,
  "8": 85422,
  "acos": 38519,
  "sinh": 27826,
  "cosh": 30871,
  "asinh": 45049,
  "asin": 43009,
  "acosh": 43578,
  "atanh": 36033,
  "atan": 38988,
  "tanh": 11742,
  "0": 79757,
  "abs": 8754,
  "sign": 5527,
  "9": 52125,
  "7": 49635,
  "E": 26619,
  "pi": 8747,
  "cot": 141,
  "acot": 14,
  "coth": 1
}


In [ ]:
# remove any lines with "cot", "acot", "coth" in them as they are low in frequency
# this will brings us to a neat 42 tokens including the SPECIAL tokens at tokenisation time!
total_lines = sum(1 for _ in open("train16.txt", 'r', encoding='utf-8'))
with open("train16.txt", "r", encoding="utf-8") as fin:
    with open("dataset2.txt", "w", encoding="utf-8") as fout:
        for line in tqdm(fin, total=total_lines, desc="Processing lines"):
            line = line.strip()
            if not line:
                continue
            if "cot" in line or "acot" in line or "coth" in line:
                continue
            # rename "a8" to "c", "ln" to "log", "Y" to "y", "Y'" to "y'" cause those look good
            line = line.replace("a8", "c").replace("ln", "log").replace("Y", "y").replace("Y'", "y'")
            fout.write(line + "\n")

tokens = {}
total_lines = sum(1 for _ in open("dataset2.txt", 'r', encoding='utf-8'))
with open("dataset2.txt", "r", encoding="utf-8") as fin:
    for line in tqdm(fin, total=total_lines, desc="Processing lines"):
        line = line.strip()
        if not line:
            continue
        for token in line.split():
            if token not in tokens:
                tokens[token] = 0
            tokens[token] += 1
print("token dict after cleaning:", json.dumps(tokens, indent=2))

Processing lines: 100%|██████████| 683446/683446 [00:03<00:00, 207129.69it/s]

token dict after cleaning: {
  "add": 2196187,
  "mul": 2676039,
  "INT-": 1162871,
  "2": 877565,
  "y": 469067,
  "x": 1893113,
  "y'": 731574,
  "c": 683702,
  "pow": 532449,
  "INT+": 1899762,
  "3": 416734,
  "1": 974800,
  "4": 373781,
  "log": 254389,
  "exp": 294014,
  "sin": 216147,
  "cos": 227196,
  "sqrt": 164894,
  "tan": 134167,
  "5": 365681,
  "6": 132617,
  "div": 103882,
  "8": 85421,
  "acos": 38518,
  "sinh": 27824,
  "cosh": 30866,
  "asinh": 45046,
  "asin": 43008,
  "acosh": 43576,
  "atanh": 36033,
  "atan": 38983,
  "tanh": 11742,
  "0": 79755,
  "abs": 8754,
  "sign": 5527,
  "9": 52125,
  "7": 49633,
  "E": 26619,
  "pi": 8741
}


In [ ]:
import sympy as sp
from sympy import Derivative
import os

from tqdm import tqdm

FILE_PATH = "dataset2.txt"
x = sp.Symbol('x', real=True, nonzero=True)
c = sp.Symbol('c', real=True)
f = sp.Function('f', real=True, nonzero=True)


def prefix_to_sympy(prefix, arity_map):
    def helper(tokens):
        # pop the first token
        token = tokens.pop(0)

        if token == 'x':
            return x
        elif token == 'c':
            return c
        elif token == 'y':
            return f(x)
        elif token == "y'":
            return Derivative(f(x), x)
        

        if token.isdigit():
            return sp.Integer(int(token))

        # mathematical constants
        if token == 'E':
            return sp.E
        if token == 'pi':
            return sp.pi

        elif token in arity_map:
            arity = arity_map[token]
            _args = []
            for _ in range(arity):
                arg = helper(tokens)
                _args.append(arg)

            # basic operators
            if token == 'add':
                return sp.Add(*_args)
            elif token == 'sub':
                return sp.Add(_args[0], sp.Mul(-1, _args[1]))
            elif token == 'mul':
                return sp.Mul(*_args)
            elif token == 'div':
                return sp.Mul(_args[0], sp.Pow(_args[1], -1))
            elif token == 'pow':
                return sp.Pow(*_args)
            elif token == 'sqrt':
                return sp.sqrt(_args[0])
            elif token == 'exp':
                return sp.exp(_args[0])
            elif token == 'log':
                return sp.log(_args[0])
            elif token == 'abs':
                return sp.Abs(_args[0])
            elif token == 'sign':
                return sp.sign(_args[0])
            elif token in ['INT-', 'INT+']:
                return sp.Integer(_args[0] * -1 if token == 'INT-' else _args[0])

            # trig operators
            elif token == 'sin':
                return sp.sin(_args[0])
            elif token == 'cos':
                return sp.cos(_args[0])
            elif token == 'tan':
                return sp.tan(_args[0])

            # inverse trig operators
            elif token == 'asin':
                return sp.asin(_args[0])
            elif token == 'acos':
                return sp.acos(_args[0])
            elif token == 'atan':
                return sp.atan(_args[0])

            # hyperbolic operators
            elif token == 'sinh':
                return sp.sinh(_args[0])
            elif token == 'cosh':
                return sp.cosh(_args[0])
            elif token == 'tanh':
                return sp.tanh(_args[0])

            # inverse hyperbolic operators
            elif token == 'asinh':
                return sp.asinh(_args[0])
            elif token == 'acosh':
                return sp.acosh(_args[0])
            elif token == 'atanh':
                return sp.atanh(_args[0])
            else:
                print(f"Unknown operator: {token}")
        else:
            print(f"Unknown token: {token}")

    tokens = list(prefix)
    return helper(tokens)


OPERATORS = {
    # basic - binary
    'add': 2, 'sub': 2, 'mul': 2, 'div': 2, 'pow': 2,
    # basic - unary
    'sqrt': 1, 'exp': 1, 'log': 1, 'abs': 1, 'sign': 1,
    # trig - unary
    'sin': 1, 'cos': 1, 'tan': 1,
    # inverse trig - unary
    'asin': 1, 'acos': 1, 'atan': 1,
    # hyperbolic - unary
    'sinh': 1, 'cosh': 1, 'tanh': 1,
    # inverse hyperbolic - unary
    'asinh': 1, 'acosh': 1, 'atanh': 1,
    # INT - unary
    'INT+': 1, 'INT-': 1,
}


# dataset sanity check
total_lines = sum(1 for _ in open(FILE_PATH, 'r', encoding='utf-8'))
with open(FILE_PATH, "r", encoding="utf-8") as fin:
    for line in tqdm(fin, total=total_lines, desc="Processing lines"):
        line = line.strip()
        if not line:
            continue
        try:
            input_expr, output_expr = line.split("\t")
            input_tokens = input_expr.strip().split()
            output_tokens = output_expr.strip().split()
            input_sympy = prefix_to_sympy(input_tokens, OPERATORS)
            output_sympy = prefix_to_sympy(output_tokens, OPERATORS)
        except Exception as e:
            print(f"Error processing line: {line}. Error: {e}")
            break
        

Processing lines: 100%|██████████| 683442/683442 [06:45<00:00, 1684.59it/s]
